In [1]:
import pandas as pd
import numpy as np
import re

import scispacy
import spacy

from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI


In [2]:
# Ustawienie opcji wyświetlania
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Wczytanie danych

In [3]:
data = pd.read_csv("C:\\Users\\justy\\Documents\\Projekt\\02_dane_dolegliwosci_po_wstepnym_preprocessingu_i_wstepnej_analizy_EDA.csv")  #wczytanie danych
data.head()

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa
0,Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,Nie,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,Nie,Nie,NaN,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,Tak,Tak,"Hypertension, brain cysts, lung tumors",4,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,40-49 lat
1,"Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.",Nie,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,NaN,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation",Tak,Tak,"hypothyroidism, chronic hepatitis",2,Nie,Do pół roku,wartość prawidłowa,0,30-39 lat
2,"Stage IV endometriosis. Adhesions, frozen pelvis. Endometrial mass on the ovary, masses in the rectum. Bowel fused with the uterus. Bowel adhered to the peritoneum. Ovaries retracted behind the uterus and fused with the uterus and bowel. Bladder displaced and tethered by adhesions. Adhesions on the parametrium, ligaments.","Laparoscopy: release from adhesions, removal of the left ovary with appendages, excision of endometrial lesions from the right ovary, removal of the right appendages. Disc-shaped transrectal resection of the rectum. Release from intestinal adhesions. Hysterectomy, cervix removal. Release from adhesions and repositioning of the bladder.",Tak,"Persistent lower back pain. Urinary system issues. Decreased urine pressure, pain in the ureter and kidney areas.",laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Nie,NaN,Tak,Tak,Byłam wielokrotnie u osteopaty/fizjoterapeuty w okresie do pół roku po operacji oraz w późniejszym okresie.,Ćwiczę 1 raz w tygodniu i/lub średnio robię więcej niż 2 tys kroków dziennie.,Tak,"Massage and cupping on the lower back; naturopathy, treatment with supplements.",Tak,Nie,NaN,1,Nie,Od 1 roku do 2 lat,nadwaga,0,30-39 lat
3,"Advanced adenomyosis, left and right parametric endometriosis, ovarian endometriosis, intra-abdominal adhesions.","Release of adhesions, hysterectomy, removal of endometriosis from the ovaries and adnexa, anti-adhesion gel, drainage of the peritoneum.",Tak,"Every few weeks, there is a sudden sharp pain (cramp) in the intestine near the ileocecal valve lasting up to an hour, accompanied by cold sweats. The pain occurs on different days of the cycle and always after one day of constipation (suspected intestinal adhesion).",laparoskopia,Nie,Nie,NaN,Tak,Nie,NaN,Ćwiczę 3 razy 

# Wyodrębnienie jednostek nazwanych NER dotyczących przetłumaczonych na angielski kolumn opisowych

W celu wyodrębnienia ujednoliconych danych z kolumn opisowych, używam modeli dedykowanych do celów biomedycznych, do wyodrębniania jednostek nazwanych NER z biblioteki scispacy.

Według informacji ze strony: https://allenai.github.io/scispacy/ :

|model	|F1	|Entity Types|Wyjaśnienie celu modelu.|Czy nadaje się do celów niniejszego projektu|
|-------|---|------------|------------------------|--------------------------------------------|
|en_ner_craft_md	|78.01	|GGP, SO, TAXON, CHEBI, GO, CL| Model ten rozpoznaje jednostki związane z genami, chemikaliami i taksonomią organizmów.| Nie|
|en_ner_jnlpba_md	|72.06	|DNA, CELL_TYPE, CELL_LINE, RNA, PROTEIN| Model ten jest przeznaczony do rozpoznawania jednostek związanych z DNA, RNA, białkami i komórkami.| Nie|
|en_ner_bc5cdr_md	|84.28	|DISEASE, CHEMICAL|Ten model specjalizuje się w wykrywaniu nazw chorób oraz substancji chemicznych, szczególnie w kontekście tekstów biomedycznych.| Tak|
|en_ner_bionlp13cg_md	|77.84	|AMINO_ACID, ANATOMICAL_SYSTEM, CANCER, CELL, CELLULAR_COMPONENT, DEVELOPING_ANATOMICAL_STRUCTURE, GENE_OR_GENE_PRODUCT, IMMATERIAL_ANATOMICAL_ENTITY, MULTI-TISSUE_STRUCTURE, ORGAN, ORGANISM, ORGANISM_SUBDIVISION, ORGANISM_SUBSTANCE, PATHOLOGICAL_FORMATION, SIMPLE_CHEMICAL, TISSUE|Model ten rozpoznaje szeroki zakres jednostek związanych z anatomią, genami, nowotworami, komórkami oraz substancjami chemicznymi.| Tak|


In [4]:
# Inicjalizacja modeli
nlp_bionlp13cg = spacy.load("en_ner_bionlp13cg_md")  # Ładowanie modelu BioNLP13CG
nlp_bc5cdr = spacy.load("en_ner_bc5cdr_md")          # Ładowanie modelu BC5CDR

c:\Users\justy\AppData\Local\Programs\Python\Python38\lib\site-packages\spacy\language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [5]:
# Funkcja przetwarzająca tekst i zwracająca listy jednostek, etykiet i modeli
def process_text(text):
    if pd.isna(text):
        return np.nan
    
    # Przetworzenie tekstu przez wszystkie modele
    doc_bionlp13cg = nlp_bionlp13cg(text)
    doc_bc5cdr = nlp_bc5cdr(text)

    # Przechowywanie wszystkich jednostek rozpoznanych przez modele
    entities = []

    # Zbiór jednostek z modelu bionlp13cg
    for ent in doc_bionlp13cg.ents:
        entities.append((ent.text, ent.label_, 'model: bionlp13cg'))

    # Zbiór jednostek z modelu bc5cdr
    for ent in doc_bc5cdr.ents:
        entities.append((ent.text, ent.label_, 'model: bc5cdr'))

    # Tworzenie list dla nowych kolumn
    texts = [ent[0] for ent in entities]
    labels = [ent[1] for ent in entities]
    models = [ent[2] for ent in entities]

    return texts, labels, models

# Lista kolumn, które chcę przetworzyć w celu wyodrębnienia jednostek nazwanych (entities)
columns_to_process = ['Tlumaczenie_Ang_Rozpoznanie_pooperacyjne', 'Tlumaczenie_Ang_Zakres_operacji', 'Tlumaczenie_Ang_Rodzaje_dolegliwosci', 'Tlumaczenie_Ang_Jakie_inne_metody', 'Tlumaczenie_Ang_Jakie_inne_choroby']

# Iteracja przez kolumny
for column in columns_to_process:
    data[[f'{column}_entities', f'{column}_entity_types', f'{column}_entity_models']] = data[column].apply(
        lambda x: pd.Series(process_text(x)) if not pd.isna(x) else pd.Series([np.nan, np.nan, np.nan])
    )

data.head()


,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_types,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_models,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji_entity_types,Tlumaczenie_Ang_Zakres_operacji_entity_models,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_types,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_models,Tlumaczenie_Ang_Jakie_inne_metody_entities,Tlumaczenie_Ang_Jakie_inne_metody_entity_types,Tlumaczenie_Ang_Jakie_inne_metody_entity_models,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby_entity_types,Tlumaczenie_Ang_Jakie_inne_choroby_entity_models
0,Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,Nie,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,Nie,Nie,NaN,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,Tak,Tak,"Hypertension, brain cysts, lung tumors",4,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,40-49 lat,"[Intra-abdominal adhesions, uterine body, left parametrium, ureter, left uterosacral ligament, ureter, uterosacral ligament, Endometrial tumor, cervix, left adnexa, Intra-abdominal adhesions, Adenomyosis of the uterine body, endometriosis, endometriosis, Endometrial tumor, Endometriosis]","[PATHOLOGICAL_FORMATION, MULTI_TISSUE_STRUCTURE, MULTI_TISSUE_STRUCTURE, PATHOLOGICAL_FORMATION, MULTI_TISSUE_STRUCTURE, ORGAN, MULTI_TISSUE_STRUCTURE, CANCER, CANCER, MULTI_TISSUE_STRUCTURE, DISEASE, DISEASE, DISEASE, DISEASE, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr]","[adhesions, cervical tumor, left, peritoneal cavity, endometriosis, tumor]","[PATHOLOGICAL_FORMATION, CANCER, TISSUE, MULTI_TISSUE_STRUCTURE, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr]",NaN,NaN,NaN,NaN,NaN,NaN,"[brain, lung tumors, Hypertension, lung tumors]","[ORGAN, CANCER, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr]"
1,"Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.",Nie,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,NaN,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średn

In [6]:
# W kolumnach z jednostkami nazwanymi zamiana wszystkich liter na małe oraz utworzenie zbiorów z unikalnymi tekstami żeby uniknąć przypadku, w którym jednostki nazwane (entities) powtarzają się.

# Funkcja do zamiany liter na małe i usuwania białych znaków
def process_list(value):
    if isinstance(value, list):
        # Zamień listę wartości na małe litery i usuń białe znaki
        processed = [str(item).lower().strip() for item in value if pd.notna(item)]
        return list(set(processed))  # Zamień listę na zestaw i z powrotem na listę unikalnych wartości
    return np.nan  # Jeśli wartość nie jest listą, zwróć nan

# Lista kolumn do przetworzenia
columns_to_process_to_unique = ['Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities', 'Tlumaczenie_Ang_Zakres_operacji_entities', 'Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities', 'Tlumaczenie_Ang_Jakie_inne_metody_entities', 'Tlumaczenie_Ang_Jakie_inne_choroby_entities']

# Iteracja przez kolumny
for column in columns_to_process_to_unique:
    data[column] = data[column].apply(process_list)


In [7]:
# Sprawdzam tylko te kolumny, które obrabiałam

# Lista kolumn oryginalnych oraz dodanych
columns_to_display = []

for column in columns_to_process:
    # Dodaj oryginalne kolumny
    columns_to_display.append(column)
    
    # Dodaj nowe kolumny
    columns_to_display.append(f'{column}_entities')
    columns_to_display.append(f'{column}_entity_types')
    columns_to_display.append(f'{column}_entity_models')

# Wyświetlenie
data[columns_to_display].head(10)

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_types,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_models,Tlumaczenie_Ang_Zakres_operacji,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji_entity_types,Tlumaczenie_Ang_Zakres_operacji_entity_models,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_types,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_models,Tlumaczenie_Ang_Jakie_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody_entities,Tlumaczenie_Ang_Jakie_inne_metody_entity_types,Tlumaczenie_Ang_Jakie_inne_metody_entity_models,Tlumaczenie_Ang_Jakie_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby_entity_types,Tlumaczenie_Ang_Jakie_inne_choroby_entity_models
0,Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,"[uterine body, left parametrium, adenomyosis of the uterine body, endometrial tumor, left adnexa, endometriosis, left uterosacral ligament, uterosacral ligament, ureter, cervix, intra-abdominal adhesions]","[PATHOLOGICAL_FORMATION, MULTI_TISSUE_STRUCTURE, MULTI_TISSUE_STRUCTURE, PATHOLOGICAL_FORMATION, MULTI_TISSUE_STRUCTURE, ORGAN, MULTI_TISSUE_STRUCTURE, CANCER, CANCER, MULTI_TISSUE_STRUCTURE, DISEASE, DISEASE, DISEASE, DISEASE, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr, model: bc5cdr]",Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,"[cervical tumor, peritoneal cavity, tumor, endometriosis, left, adhesions]","[PATHOLOGICAL_FORMATION, CANCER, TISSUE, MULTI_TISSUE_STRUCTURE, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hypertension, brain cysts, lung tumors","[hypertension, lung tumors, brain]","[ORGAN, CANCER, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr]"
1,"Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","[peritoneal, abdomen, endometriosis, peritoneal diseases, pain]","[PATHOLOGICAL_FORMATION, MULTI_TISSUE_STRUCTURE, DISEASE, DISEASE, DISEASE]","[model: bionlp13cg, model: bionlp13cg, model: bc5cdr, model: bc5cdr, model: bc5cdr]","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.","[abdominal cavity, peritoneal tissues, left sacrouterine ligament, adhesions, anterior wall, peritoneum]","[MULTI_TISSUE_STRUCTURE, CELLULAR_COMPONENT, MULTI_TISSUE_STRUCTURE, MULTI_TISSUE_STRUCTURE, MULTI_TISSUE_STRUCTURE, MULTI_TISSUE_STRUCTURE, PATHOLOGICAL_FORMATION]","[model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg, model: bionlp13cg]",NaN,NaN,NaN,NaN,"psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation",[],[],[],"hypothyroidism, chronic hepatitis","[hypothyroidism, chronic hepatitis]","[DISEASE, DISEASE]","[model: bc5cdr, model: bc5cdr]"
2,"Stage IV endometriosis. Adhesions, frozen pelvis.

In [8]:
# Sprawdzam jakie są typy danych w wybranej nowej kolumnie z jednostkami nazwanymi (entities)
data['Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities'].apply(type).unique()


array([<class 'float'>, <class 'list'>], dtype=object)

In [9]:
# W wybranej kolumnie są dwa typy danych: float i list. Sprawdzam jeszcze jak to wygląda:
data[['Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities']].assign(typ_danych=data['Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities'].apply(type))


,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,typ_danych
0,NaN,<class 'float'>
1,NaN,<class 'float'>
2,"[urine, kidney, ureter, lower back pain, pain]",<class 'list'>
3,"[intestinal, ileocecal valve, constipation, intestine, cramp, pain]",<class 'list'>
4,NaN,<class 'float'>
5,"[lower back pain, pain]",<class 'list'>
6,"[lower abdominal, abdominal pain]",<class 'list'>
7,"[groin, bladder, abdomen, left, pelvis, painful urination, pain]",<class 'list'>
8,NaN,<class 'float'>
9,[bladder],<class 'list'>


Typy danych są prawidłowe.

Dla kolumny 'Tlumaczenie_Ang_Jakie_inne_metody' nie udało się wyodrębnić jednostek nazwanych. Usunę kolumny dodane funkcją process_text do tej kolumny.

Jednostki nazwane wyodrębnione dla pozostałych kolumn nie tworzą w pełni zadowalających list, aczkolwiek mogą być przydatne do późniejszej analizy. Zostawię kolumny z nazwami wyodrębnionych jednostek (entities), ale kolumny z typami jednostek i nazwami modeli usunę, gdyż nie są do niczego potrzebne, a służyły jedynie do sprawdzenia danych.


In [10]:
#Kolumny dataframe:
cols = list(data.columns)

# Przeniesienie kolumn z jednostkami nazwanymi (entities) przed ich odpowiadającą kolumną
cols.insert(cols.index('Tlumaczenie_Ang_Rozpoznanie_pooperacyjne'), cols.pop(cols.index('Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities')))
cols.insert(cols.index('Tlumaczenie_Ang_Zakres_operacji'), cols.pop(cols.index('Tlumaczenie_Ang_Zakres_operacji_entities')))
cols.insert(cols.index('Tlumaczenie_Ang_Rodzaje_dolegliwosci'), cols.pop(cols.index('Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities')))
cols.insert(cols.index('Tlumaczenie_Ang_Jakie_inne_choroby'), cols.pop(cols.index('Tlumaczenie_Ang_Jakie_inne_choroby_entities')))

# Przestawienie DataFrame według zaktualizowanej listy kolumn:
data = data[cols]

# Usunięcie dodatkowych niepotrzebnych kolumn
columns_to_drop = ['Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_types', 'Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entity_models', 'Tlumaczenie_Ang_Zakres_operacji_entity_types', 'Tlumaczenie_Ang_Zakres_operacji_entity_models', 'Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_types', 'Tlumaczenie_Ang_Rodzaje_dolegliwosci_entity_models', 'Tlumaczenie_Ang_Jakie_inne_metody_entities', 'Tlumaczenie_Ang_Jakie_inne_metody_entity_types', 'Tlumaczenie_Ang_Jakie_inne_metody_entity_models', 'Tlumaczenie_Ang_Jakie_inne_choroby_entity_types',	'Tlumaczenie_Ang_Jakie_inne_choroby_entity_models']
data = data.drop(columns=columns_to_drop)
data.head()

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa
0,"[uterine body, left parametrium, adenomyosis of the uterine body, endometrial tumor, left adnexa, endometriosis, left uterosacral ligament, uterosacral ligament, ureter, cervix, intra-abdominal adhesions]",Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,"[cervical tumor, peritoneal cavity, tumor, endometriosis, left, adhesions]",Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,Nie,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,Nie,Nie,NaN,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,Tak,Tak,"[hypertension, lung tumors, brain]","Hypertension, brain cysts, lung tumors",4,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,40-49 lat
1,"[peritoneal, abdomen, endometriosis, peritoneal diseases, pain]","Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","[abdominal cavity, peritoneal tissues, left sacrouterine ligament, adhesions, anterior wall, peritoneum]","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.",Nie,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,NaN,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation",Tak,Tak,"[hypothyroidism, chronic hepatitis]","hypothyroidism, chronic hepatitis",2,Nie,Do pół roku,wartość prawidłowa,0,30-39 lat
2,"[bladder, ligaments, parametrium, bowel, ovary, ovaries, rectum, uterus, endometriosis, adhesions, endometrial, masses, peritoneum]","Stage IV endometriosis. Adhesions, frozen pelvis. Endometrial mass on the ovary, masses in the rectum. Bowel fused with the uterus. Bowel adhered to the peritoneum. Ovaries retracted behind the uterus and fused with the uterus and bowel. Bladder displaced and tethered by adhesions. Adhesions on the parametrium, ligaments.","[bladder, cervix, ovary, rectum, intestinal adhesions, adhesions, endometrial lesions]","Laparoscopy: release from adhesions, removal of the left ovary with appendages, excision of endometrial lesions from the right ovary, removal of the right appendages. Disc-shaped transrectal resection of the rectum. Release from intestinal adhesions. Hysterectomy, cervix removal. Release from adhesions and repositioning of the bladder.",Tak,"[urine, kidney, ureter, lower back pain, pain]","Persistent lower back pain. Urinary system issues. Decreased urine pressure, pain in the ureter and kidne

Wyodrębnienie jednostek nazwanych NER przyniosło dość dobry rezultat, jednak niewystarczająco zadowalający, a dodatkowo dla kolumny 'Tlumaczenie_Ang_Jakie_inne_metody' wcale nie udało się ich wyodrębnić. W związku z tym w następnym kroku będzie wykorzystanie LLM do próby wyodrębnienia z tekstów bardziej ustrukturyzowanych danych.

Za pomocą LLM dla każdej z kolumn zostaną wyodrębnione listy z najważniejszymi informacjami. Na przykład dla kolumny 'Tlumaczenie_Ang_Rodzaje_dolegliwosci' zostanie wyodrębniona lista dolegliwości, itd. dla kolejnych kolumn.

# LLM

-> Wykorzystanie LLM do wyciągnięcia najważniejszych i najistotniejszych informacji z kolumn opisowych.

Pytania opisowe: Model musi wychwycić najważniejsze informacje. Opisy osób, które wypełniły ankiety są bardzo różnorodne. 
Poniżej wykonam kroki:
- użycie LLM do wyciągnięcia w syntetyczny sposób to co jest w opisowych kolumnach czyli rozpoznanie, co zrobiono, dolegliwości, schorzenia, stosowane alternatywne metody wspomagania organizmu,
- zapisanie do listy,
- stworzenie nowej kolumny z listami dla każdego wiersza.

In [ ]:
api_key = open("C:\\...\\api openai key.txt").read()	# w miejsce ... ma być lokalizacja
model_1 = ChatOpenAI(model='gpt-4o', temperature=0.0, openai_api_key=api_key)

Utworzenie klasy CustomOutputParser

In [13]:
# Ta klasa do wyciągania tylko potrzebnych informacji z outputu będzie wykorzystana również w dalszej części programu, gdy będę wyciągać listę dolegliwości
class CustomOutputParser:
	def parse(self, content):
		lines = content.split('\n')
		result = []

		# Przechodzimy przez linie i ignorujemy wprowadzenie
		for line in lines:
			# Pomijamy puste linie i linie będące wprowadzeniem
			if line.strip() == '' or line.endswith(':'):
				continue
			# Usuwamy znaki nielistowe z początku linii
			if line.startswith('* '):
				line = line[2:]
			result.append(line.strip())

		return result

## Wyciągnięcie listy dolegliwości (dla kolumny 'Tlumaczenie_Ang_Rodzaje_dolegliwosci')

In [14]:
# Tworzę system template.

system_template = """
You are an experienced doctor of medical science and have received filled surveys from your female patients. 
Patients described their symptoms.
Your task is to create a medical report listing only the actual symptoms described by the patients, in short form, excluding any speculations or causes mentioned by them.
Patients sometimes, instead of writing only about symptoms, also write that something happened in the past but is no longer true now. Skip such additions.
Please list these symptoms as individual items in a list, omitting any patient conjecture. The output should be a list of all identified symptoms.
If the text contains generic responses such as "yes," "no," or any similar phrases that do not describe specific ailments, return "No symptoms provided."
Anxiety is also a symptom.

For example:
Patients description: ["Patient_1 describes ailment 1, then talks about past events, then writes about ailment 2, then writes what he thinks might be the cause of these ailments.", "Patient_2 describes his ailment_1", "Patient_3 wrote only 'Yes'", "Patient_4 describes his ailment 1, then his ailment 2, and then still his ailment 3, which is the same as patient_1's ailment 2."]
Output:
["symptom 1", "symptom 2", "symptom 3", "No symptoms provided", "symptom 4", "symptom 5"]

You have to extract all symptoms without unnecessary descriptions and make a list of them. Don't repeat the symptoms. Each symptom should be a short phrase or word, no more than 4 words!

Avoid any extra descriptions. List each symptom as a comma-separated value, without additional explanation or text.
Be mindful of patients describing their symptoms with grammatical and syntactical errors. Do not duplicate these errors.

Symptoms should be identified only from descriptions provided by patients. Be accurate. Do not over-interpret. Keep in mind that anxiety therapy and visiting doctors are not ailments or symptoms!

"""

In [15]:
human_template = """Filled surveys about female patients' descriptions: {symptoms}. Identify and extract ALL ailments or symptoms from the text. You always reply to questions only as a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz'. Return ONLY this comma separated list, no other words!"""


system_prompt = SystemMessagePromptTemplate.from_template(system_template) 
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

system_prompt.input_variables

human_prompt.input_variables

chat_template.input_variables


# Inicjalizacja parsera
output_parser = CustomOutputParser()

# Funkcja do przetwarzania objawów z obsługą NaN
def process_symptom_description(description):
	if isinstance(description, list):
		description = ' '.join(description)
	if pd.isna(description):
		return np.nan
	try:
		prompt = chat_template.format_prompt(topic=description, symptoms=description)
		response_list = []
		for chunk in model_1.stream(prompt.to_messages()):
			response_list.append(chunk.content)
		# Łączenie fragmentów odpowiedzi w jedno
		response_text = ''.join(response_list).replace('\n', ' ').strip()

		return response_text
	
	except Exception as e:
		print(f"Error processing description '{description}': {e}")
		return np.nan 

# Tworzenie nowej kolumny z przetworzonymi wynikami
data['Wyodrebnione_dolegliwosci'] = data['Tlumaczenie_Ang_Rodzaje_dolegliwosci'].apply(process_symptom_description)


In [16]:
# Sprawdzam wybrane kilka wierszy:
data.iloc[14:18]

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Wyodrebnione_dolegliwosci
14,"[ovary, foci, endometrial cyst, adhesions, cyst of the right ovary, peritoneum]","Adhesions, multiple foci in the peritoneum, endometrial cyst of the right ovary.","[laparotomy - cyst excision, laparotomy - cyst]","Laparotomy - cyst excision, adhesiolysis",Tak,"[pain, abdomen]","Adhesion pain. I was also stitched too tightly, which causes quite frequent discomfort throughout the abdomen, where there is simply no space.",laparotomia,Tak -> ciągle do jednego roku po operacji,Nie,NaN,Tak,Tak,Byłam wielokrotnie u osteopaty/fizjoterapeuty w okresie do pół roku po operacji oraz w późniejszym okresie.,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,Tak,Tak,"[anxiety, insulin, depression]","insulin resistance, depression, anxiety disorders",2,Nie,Od pół roku do 1 roku,wartość prawidłowa,0,20-29 lat,"Adhesion pain, frequent discomfort"
15,"[stomach pain, stomach, legs, ovary, right ovary pain, lower back pain, pain]","Lower back pain, stomach pain, left and right ovary pain radiating to the legs, generally a feeling of pain in all the cavities.","[abdominal wall, tumor, uterus, left, left fallopian, growths, intestines, polyps]","Deeply infiltrating, stage 4, removal of growths on the left side, removal of a tumor on the intestines, polyps, removal of the left fallopian tube, the body of the uterus adheres to the abdominal wall.",Tak,[],Yes,laparoskopia,Tak -> ciągle do pół roku po operacji,Nie,NaN,Tak,Nie,NaN,Ćwiczę 1 raz w tygodniu i/lub średnio robię więcej niż 2 tys kroków dziennie.,Nie,NaN,Tak,Nie,NaN,NaN,1,Nie,Powyżej 3 lat,wartość prawidłowa,0,30-39 lat,No symptoms provided
16,[],"Cyst of the right adnexa, adhesions","[right adnexal cyst, adnexal cyst]","Laparoscopy for adhesiolysis, excision of right adnexal cyst.",Nie,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Nie,NaN,Tak,Nie,NaN,W ogóle nie jestem aktywna fizycznie i prowadzę siedzący tryb życia.,Nie,NaN,Nie,Tak,[depression],Depression,2,Nie,Do pół roku,nadwaga,0,30-39 lat,NaN
17,"[sacrouterine ligament, abdominal cavity, sacral bone, foci, polyp, uterus, ovarian cyst, adhesions, intestines]","adhesions on the sacrouterine ligament, ovarian cyst, infiltrates and adhesions, other inflammatory foci in the abdominal cavity and sacral bone, intestines occluded with inflammatory foci, polyp in the uterus",[],Laparoscopic release and removal of the above changes.,Nie,NaN,NaN,laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Nie,NaN,Tak,Tak,Byłam wielokrotnie u osteopaty/fizjoterapeuty w okresie do pół roku po operacji oraz w późniejszym okresie.,Ćwiczę 2 razy w tygodniu i/lub średnio robię więcej niż 5 tys kroków dziennie.,Nie,NaN,Nie,Tak,[depression],Depression,1,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,30-39 lat,NaN


In [17]:
data.tail()

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Wyodrebnione_dolegliwosci
52,[left ovarian endometrial cyst],Left ovarian endometrial cyst,[endometrial cyst],Removal of the endometrial cyst (only),Tak,[cyst],"Three months after the surgery, the cyst has recurred.",laparoskopia,Nie,Nie,NaN,Tak,Nie,NaN,Ćwiczę 1 raz w tygodniu i/lub średnio robię więcej niż 2 tys kroków dziennie.,Tak,Relaxation,Tak,Tak,"[hemorrhoids, sarcoidosis, neurosis, hyperthyroidism]","Hyperthyroidism, hemorrhoids, sarcoidosis, I also had neurosis.",1,Nie,Powyżej 3 lat,wartość prawidłowa,0,30-39 lat,No symptoms provided
53,"[adenomyosis, ovary, endometriosis, uterus, adnexa, intestines]","Endometriosis and adenomyosis. \nLaparotomy, removal of the body of the uterus along with the adnexa and the left ovary. Release of the intestines.","[uterus, intestines, ovary]","Laparotomy, removal of the body of the uterus along with the appendages and the left ovary. Release of the intestines.",Nie,NaN,NaN,laparotomia,Tak -> ciągle do jednego roku po operacji,Nie,NaN,Tak,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,W ogóle nie jestem aktywna fizycznie i prowadzę siedzący tryb życia.,Tak,"Psychologist, breathing exercises",Tak,Nie,NaN,NaN,1,Nie,Od 1 roku do 2 lat,wartość prawidłowa,1,30-39 lat,NaN
54,"[adenomyosis, tumor, rectum, umbilicus, endometriosis, adhesions, cyst]","Endometriosis stage IV with deep infiltration, adenomyosis, tumor of the rectum, tumor of the small and large intestine, adhesions, cyst in the umbilicus.","[adenomyosis, tumor, endometriosis, bowel tumor, adhesions, stage iv deep infiltrating, rectal tumor]","Laparoscopy with mini laparotomy: Stage IV deep infiltrating endometriosis, adenomyosis, rectal tumor, small and large bowel tumor, adhesions, umbilical cyst.",Tak,"[bathroom and bowel movements, bowel, constipation, anxiety, nausea, mood swings]","Feeling of incomplete bowel evacuation, constipation about six months after surgery, ended twice with visits to the emergency room, nausea, mood swings, menopause, hot flashes, chronic anxiety following the illness related to going to the bathroom and bowel movements, visits to a psychiatrist, anxiety treatment therapy.",minilaparotomia,Tak -> ciągle powyżej dwóch lat po operacji,Tak,Ciągle do jednego roku po operacji,Tak,Nie,NaN,W ogóle nie jestem aktywna fizycznie i prowadzę siedzący tryb życia.,Tak,Visits to a psychotherapist and psychiatrist,Tak,Tak,"[anxiety, hashimoto, psychiatric]","Hashimoto, chronic anxiety post-surgery, I am receiving psychiatric treatment.",1,Nie,Od 2 lat do 3 lat,nadwaga,0,30-39 lat,"incomplete bowel evacuation, constipation, nausea, mood swings, hot flashes, chronic anxiety"
55,"[left parametrium, tumor, sacrouterine ligament, endometriosis - stage 4, left ovarian cyst, endometriosis, diaphragm, sigmoid colon, ovarian cyst, intra-abdominal adhesions, tumors]","Deep infiltrating endometriosis of the left parametrium, intra-abdominal adhesions, advanced endometriosis - stage 4, left ovarian cyst, adhesion of the sigmoid colon, tumor on the sacrouterine ligament, tumors on the diaphragm.","[tumor, sacrouterine ligament, ovary, left ovarian cyst, diaphragm, sigmoid colon, adhesions, tumors]","Laparoscopy: Release of adhesions, removal of the left ovarian cyst, separation of the sigmoid 

## Wyciągnięcie listy rozpoznań pooperacyjnych (dla kolumny 'Tlumaczenie_Ang_Rozpoznanie_pooperacyjne')

In [18]:
system_template = """
You are an experienced medical doctor and have received completed questionnaires from your female patients. 
The patients have entered a post-operative diagnosis.
Your task is to create a medical report, listing the post-operative diagnoses, in short form, excluding any speculation or reasons mentioned by the patients.
The result should be a list of all identified postoperative diagnoses.

You need to extract all postoperative diagnoses without unnecessary descriptions and create a list of them.

Identify postoperative diagnoses only from the descriptions provided by patients. Be accurate. Do not over-interpret.

"""

In [19]:
human_template = """Filled surveys about female patients' descriptions: {diagnoses}. Identify and extract ALL postoperative diagnoses from the text. Avoid additional descriptions. List each postoperative diagnosis as a comma-separated value, without additional explanation or text."""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

system_prompt.input_variables

human_prompt.input_variables

chat_template.input_variables

# Funkcja do przetwarzania objawów z obsługą NaN
def process_symptom_description(description):
	if isinstance(description, list):
		description = ' '.join(description)
	if pd.isna(description):
		return np.nan
	try:
		prompt = chat_template.format_prompt(topic=description, diagnoses=description)
		response_list = []
		for chunk in model_1.stream(prompt.to_messages()):
			response_list.append(chunk.content)
		# Łączenie fragmentów odpowiedzi w jedno
		response_text = ''.join(response_list).replace('\n', ' ').strip()

		return response_text
	
	except Exception as e:
		print(f"Error processing description '{description}': {e}")
		return np.nan 

# Tworzenie nowej kolumny z przetworzonymi wynikami
data['Wyodrebnione_rozpoznania_pooperacyjne'] = data['Tlumaczenie_Ang_Rozpoznanie_pooperacyjne'].apply(process_symptom_description)

In [20]:
data.head()

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Wyodrebnione_dolegliwosci,Wyodrebnione_rozpoznania_pooperacyjne
0,"[uterine body, left parametrium, adenomyosis of the uterine body, endometrial tumor, left adnexa, endometriosis, left uterosacral ligament, uterosacral ligament, ureter, cervix, intra-abdominal adhesions]",Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,"[cervical tumor, peritoneal cavity, tumor, endometriosis, left, adhesions]",Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,Nie,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,Nie,Nie,NaN,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,Tak,Tak,"[hypertension, lung tumors, brain]","Hypertension, brain cysts, lung tumors",4,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,40-49 lat,NaN,"Intra-abdominal adhesions, Adenomyosis of the uterine body, Deep infiltrating endometriosis of the left parametrium, Right parametric endometriosis, Endometrial tumor at the cervix, Endometriosis of the left adnexa"
1,"[peritoneal, abdomen, endometriosis, peritoneal diseases, pain]","Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","[abdominal cavity, peritoneal tissues, left sacrouterine ligament, adhesions, anterior wall, peritoneum]","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.",Nie,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,NaN,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation",Tak,Tak,"[hypothyroidism, chronic hepatitis]","hypothyroidism, chronic hepatitis",2,Nie,Do pół roku,wartość prawidłowa,0,30-39 lat,NaN,"Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis"
2,"[bladder, ligaments, parametrium, bowel, ovary, ovaries, rectum, uterus, endometriosis, adhesions, endometrial, masses, peritoneum]","Stage IV endometriosis. Adhesions, frozen pelvis. Endometrial mass on the ovary, masses in the rectum. Bowel fused with the uterus. Bowel adhered to the peritoneum. Ovaries retracted behind the uterus and fused with the uterus and bowel. Bladder displaced and tethered by adhesions. Adhesions on the parametrium, ligaments.","[bladder, cervix, ovary, rectum, intestinal adhesions, adhesions, endometrial lesions]","Laparoscopy: release from adhesions, removal of the left ovary with appendages, excision of endometrial lesions from the

## Wyciągnięcie list zakresów operacji (dla kolumny 'Tlumaczenie_Ang_Zakres_operacji')

In [21]:
system_template = """
You are an experienced doctor and have received completed questionnaires from your patients. 
The patients have given the scope of the operation (what was done during the operation).
Your task is to create a medical report with a short list covering the scope of the operation (what was done during the operation), excluding any speculation or reasons mentioned by the patients.
The result should be a list of all identified surgical scopes.

If the text contains only one word: “Endometriosis” return exactly that word.

Surgical scopes (what was done during surgery) should be identified only from descriptions provided by patients. Be accurate. Do not over-interpret.

"""

In [22]:
human_template = """Completed questionnaires on patient descriptions: {operation}. Identify and extract exactly from the text what was done during the patients' endometriosis surgery. Avoid additional descriptions. List each action performed as a comma-separated value, without additional explanation or text."""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

system_prompt.input_variables

human_prompt.input_variables

chat_template.input_variables

# Funkcja do przetwarzania objawów z obsługą NaN
def process_symptom_description(description):
	if isinstance(description, list):
		description = ' '.join(description)
	if pd.isna(description):
		return np.nan
	try:
		prompt = chat_template.format_prompt(topic=description, operation=description)
		response_list = []
		for chunk in model_1.stream(prompt.to_messages()):
			response_list.append(chunk.content)
		# Łączenie fragmentów odpowiedzi w jedno
		response_text = ''.join(response_list).replace('\n', ' ').strip()

		return response_text
	
	except Exception as e:
		print(f"Error processing description '{description}': {e}")
		return np.nan 

# Tworzenie nowej kolumny z przetworzonymi wynikami
data['Wyodrebnione_zakres_operacji'] = data['Tlumaczenie_Ang_Zakres_operacji'].apply(process_symptom_description)

In [23]:
data.iloc[35:44]

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Wyodrebnione_dolegliwosci,Wyodrebnione_rozpoznania_pooperacyjne,Wyodrebnione_zakres_operacji
35,"[bladder, ovary, endometrial cyst, endometrial, peritoneum]","Endometrial cyst of the left ovary, endometrial changes on the bladder and peritoneum.","[bladder, left ovarian tumor, ovary, peritoneum]","Excision of the left ovarian tumor along with the removal of the left ovary, excision of larger lesions on the bladder and peritoneum.",Tak,"[ovary, fatigue, pain]","Fatigue, pain in the area of the left ovary that has been removed, not related to menstruation. Severe pain in the lumbar region.",laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Tak,Ciągle do pół roku po operacji,Tak,Nie,NaN,Ćwiczę 2 razy w tygodniu i/lub średnio robię więcej niż 5 tys kroków dziennie.,Tak,"Yoga, meditation",Nie,Tak,"[helicobacter pylori, sibo, asthma]","SIBO, Helicobacter pylori, asthma.",1,Nie,Powyżej 3 lat,wartość prawidłowa,0,20-29 lat,"Fatigue,pain in the left ovary area,severe pain in the lumbar region","Endometrial cyst of the left ovary, endometrial changes on the bladder, endometrial changes on the peritoneum","Excision of the left ovarian tumor, removal of the left ovary, excision of larger lesions on the bladder, excision of larger lesions on the peritoneum"
36,"[bladder, infiltrating endometriosis, ovary, rectum, uterus, sigmoid colon, intestine, adhesions, intestines]","Infiltrating endometriosis affecting the intestines and bladder, with limited examination due to a significant amount of adhesions along the sigmoid colon, rectum, left ovary, uterus, bladder, right ovary, and intestine. This was solely a diagnostic laparoscopy.",[nic],"Nic, diagnostic laparoscopy.",Tak,"[loss of sensation, gallbladder, lung, adhesions, thigh, rectus, pain]","3 years of loss of sensation in the rectus and transverse abdominal muscles, with persistent pain in the area of adhesions radiating to the right thigh, gallbladder, and right lung. During laparoscopy, I was infected internally, and the treatment lasted 4 months.",laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Nie,NaN,Tak,Tak,Byłam tylko jeden raz u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"I have a profession in social rehabilitation, so I am professionally prepared for self-care and basic physiotherapy. I have developed an individual set of physiotherapy exercises and activities based on my symptoms. Due to IBS, I avoid irritating my intestines, which is why I follow a low FODMAP and low-fat diet. I avoid allergens because I also have several allergies. I avoid sitting for more than 1 hour at a time or 5 hours a day, as this triggers pain and intestinal obstructions due to adhesions. I also avoid intense physical activity for the same reason.",Nie,Tak,"[pancreatitis, fodmap sugar, hypothyroidism, ibs, multiple allergies]","Hypothyroidism, IBS, chronic pancreatitis, FODMAP sugar intolerance, multiple allergies.",1,Tak,Powyżej 3 lat,nadwaga,0,40-49 lat,"loss of sensation, persistent pain, radiating pain","Infiltrating endometriosis affecting the intestines, Infiltrating endometriosis affecting the bladder",diagnostic laparoscopy
37,"[bladder, intra-abdominal, endometriotic cysts, ovaries

## Wyciągnięcie list dodatkowych metod stosowanych przez osoby chore (dla kolumny 'Tlumaczenie_Ang_Jakie_inne_metody')

In [ ]:
system_template = """
You are an experienced doctor and have received completed questionnaires from your patients. 
Patients have stated what methods they use to improve their health or well-being, or what alternative treatments they use.
Your task is to create a medical report with a short list covering all the methods your patients use, excluding any speculation or additional descriptions.
The result should be a list of all identified methods used by patients.

If the text contains only one word, for example: "Neurologist", return exactly that word.

Methods should be identified only from descriptions provided by patients. Be accurate. Do not over-interpret.

"""

In [25]:
human_template = """Completed questionnaires on patient descriptions: {methods}. Identify and extract exactly from the text what methods patients use to improve their health or well-being or what alternative treatments they use. Avoid additional descriptions. List each method as a comma-separated value, without additional explanation or text."""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

system_prompt.input_variables

human_prompt.input_variables

chat_template.input_variables

# Funkcja do przetwarzania objawów z obsługą NaN
def process_symptom_description(description):
	if isinstance(description, list):
		description = ' '.join(description)
	if pd.isna(description):
		return np.nan
	try:
		prompt = chat_template.format_prompt(topic=description, methods=description)
		response_list = []
		for chunk in model_1.stream(prompt.to_messages()):
			response_list.append(chunk.content)
		# Łączenie fragmentów odpowiedzi w jedno
		response_text = ''.join(response_list).replace('\n', ' ').strip()

		return response_text
	
	except Exception as e:
		print(f"Error processing description '{description}': {e}")
		return np.nan 

# Tworzenie nowej kolumny z przetworzonymi wynikami
data['Wyodrebnione_dodatkowe_metody'] = data['Tlumaczenie_Ang_Jakie_inne_metody'].apply(process_symptom_description)

In [26]:
data.iloc[35:56]

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa,Wyodrebnione_dolegliwosci,Wyodrebnione_rozpoznania_pooperacyjne,Wyodrebnione_zakres_operacji,Wyodrebnione_dodatkowe_metody
35,"[bladder, ovary, endometrial cyst, endometrial, peritoneum]","Endometrial cyst of the left ovary, endometrial changes on the bladder and peritoneum.","[bladder, left ovarian tumor, ovary, peritoneum]","Excision of the left ovarian tumor along with the removal of the left ovary, excision of larger lesions on the bladder and peritoneum.",Tak,"[ovary, fatigue, pain]","Fatigue, pain in the area of the left ovary that has been removed, not related to menstruation. Severe pain in the lumbar region.",laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Tak,Ciągle do pół roku po operacji,Tak,Nie,NaN,Ćwiczę 2 razy w tygodniu i/lub średnio robię więcej niż 5 tys kroków dziennie.,Tak,"Yoga, meditation",Nie,Tak,"[helicobacter pylori, sibo, asthma]","SIBO, Helicobacter pylori, asthma.",1,Nie,Powyżej 3 lat,wartość prawidłowa,0,20-29 lat,"Fatigue,pain in the left ovary area,severe pain in the lumbar region","Endometrial cyst of the left ovary, endometrial changes on the bladder, endometrial changes on the peritoneum","Excision of the left ovarian tumor, removal of the left ovary, excision of larger lesions on the bladder, excision of larger lesions on the peritoneum","Yoga, meditation"
36,"[bladder, infiltrating endometriosis, ovary, rectum, uterus, sigmoid colon, intestine, adhesions, intestines]","Infiltrating endometriosis affecting the intestines and bladder, with limited examination due to a significant amount of adhesions along the sigmoid colon, rectum, left ovary, uterus, bladder, right ovary, and intestine. This was solely a diagnostic laparoscopy.",[nic],"Nic, diagnostic laparoscopy.",Tak,"[loss of sensation, gallbladder, lung, adhesions, thigh, rectus, pain]","3 years of loss of sensation in the rectus and transverse abdominal muscles, with persistent pain in the area of adhesions radiating to the right thigh, gallbladder, and right lung. During laparoscopy, I was infected internally, and the treatment lasted 4 months.",laparoskopia,Tak -> ciągle powyżej dwóch lat po operacji,Nie,NaN,Tak,Tak,Byłam tylko jeden raz u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"I have a profession in social rehabilitation, so I am professionally prepared for self-care and basic physiotherapy. I have developed an individual set of physiotherapy exercises and activities based on my symptoms. Due to IBS, I avoid irritating my intestines, which is why I follow a low FODMAP and low-fat diet. I avoid allergens because I also have several allergies. I avoid sitting for more than 1 hour at a time or 5 hours a day, as this triggers pain and intestinal obstructions due to adhesions. I also avoid intense physical activity for the same reason.",Nie,Tak,"[pancreatitis, fodmap sugar, hypothyroidism, ibs, multiple allergies]","Hypothyroidism, IBS, chronic pancreatitis, FODMAP sugar intolerance, multiple allergies.",1,Tak,Powyżej 3 lat,nadwaga,0,40-49 lat,"loss of sensation, persistent pain, radiating pain","Infiltrating endometriosis affecting the intestines, Infiltrating endometriosis affecting the bladder",diagnostic laparoscopy,"Self-care

## Wyciągnięcie list dodatkowych chorób i zaburzeń (dla kolumny 'Tlumaczenie_Ang_Jakie_inne_choroby')

In [27]:
system_template = """
You are an experienced medical doctor and have received completed questionnaires from your female patients. 
The patients have described their diseases and disorders.
Your task is to create a medical report in English, containing only the actual diseases and disorders described by your patients, in short form, excluding any speculation or causes mentioned by your patients.
Your patients usually speak Polish, but sometimes use other languages.
Give your answer in English. List these diseases and disorders as individual items on the list in English, omitting any patient speculation. The result should be a list of all identified diseases and disorders.

Only identify diseases and disorders based on descriptions provided by patients. Be accurate. Do not over-interpret.

"""

In [28]:
human_template = """Completed questionnaires on patient descriptions: {diseases}. Identify and extract ALL diseases and disorders from the text. Each disease or disorder should be a short phrase or word, no more than 4 words! Avoid additional descriptions. List each disease or disorder as a comma-separated value, without additional explanation or text."""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

system_prompt.input_variables

human_prompt.input_variables

chat_template.input_variables

# Funkcja do przetwarzania objawów z obsługą NaN
def process_symptom_description(description):
	if isinstance(description, list):
		description = ' '.join(description)
	if pd.isna(description):
		return np.nan
	try:
		prompt = chat_template.format_prompt(topic=description, diseases=description)
		response_list = []
		for chunk in model_1.stream(prompt.to_messages()):
			response_list.append(chunk.content)
		# Łączenie fragmentów odpowiedzi w jedno
		response_text = ''.join(response_list).replace('\n', ' ').strip()

		return response_text
	
	except Exception as e:
		print(f"Error processing description '{description}': {e}")
		return np.nan 

# Tworzenie nowej kolumny z przetworzonymi wynikami
data['Wyodrebnione_dodatkowe_choroby_i_zaburzenia'] = data['Tlumaczenie_Ang_Jakie_inne_choroby'].apply(process_symptom_description)

In [29]:
# Nowe kolumny są dodawane na końcu dataframe, więc dla lepszej czytelności dla siebie przestawiam je obok kolumn macierzystych (tych z których była ekstrakcja informacji).

#Kolumny dataframe:
cols = list(data.columns)

# Przeniesienie kolumn z wyestrahowanymi informacjami przed ich odpowiadającą kolumną
cols.insert(cols.index('Tlumaczenie_Ang_Rozpoznanie_pooperacyjne'), cols.pop(cols.index('Wyodrebnione_rozpoznania_pooperacyjne')))
cols.insert(cols.index('Tlumaczenie_Ang_Zakres_operacji'), cols.pop(cols.index('Wyodrebnione_zakres_operacji')))
cols.insert(cols.index('Tlumaczenie_Ang_Rodzaje_dolegliwosci'), cols.pop(cols.index('Wyodrebnione_dolegliwosci')))
cols.insert(cols.index('Tlumaczenie_Ang_Jakie_inne_metody'), cols.pop(cols.index('Wyodrebnione_dodatkowe_metody')))
cols.insert(cols.index('Tlumaczenie_Ang_Jakie_inne_choroby'), cols.pop(cols.index('Wyodrebnione_dodatkowe_choroby_i_zaburzenia')))

# Przestawienie DataFrame według zaktualizowanej listy kolumn:
data = data[cols]

In [30]:
# Sprawdzam czy wszystko jest dobrze, tak jak powinno
data.head()

,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne_entities,Wyodrebnione_rozpoznania_pooperacyjne,Tlumaczenie_Ang_Rozpoznanie_pooperacyjne,Tlumaczenie_Ang_Zakres_operacji_entities,Wyodrebnione_zakres_operacji,Tlumaczenie_Ang_Zakres_operacji,Czy_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities,Wyodrebnione_dolegliwosci,Tlumaczenie_Ang_Rodzaje_dolegliwosci,Rodzaj_operacji,Czy_leczenie_hormonalne,Czy_dietetyk,Okres_dietetyk,Czy_dieta_własna,Czy_Fizjoterapeuta_po_operacji,Okres_fizjoterapeuta,Aktywnosc_fizyczna,Czy_inne_metody,Wyodrebnione_dodatkowe_metody,Tlumaczenie_Ang_Jakie_inne_metody,Czy_warzywa_i_owoce_5_porcji,Czy_inne_choroby,Tlumaczenie_Ang_Jakie_inne_choroby_entities,Wyodrebnione_dodatkowe_choroby_i_zaburzenia,Tlumaczenie_Ang_Jakie_inne_choroby,Ile_operacji,Czy_operacja_zwiadowcza,Ile_po_operacji,BMI_kategoria,"Zdrowe_srodowisko (0 Nie, 1 Tak)",Grupa_wiekowa
0,"[uterine body, left parametrium, adenomyosis of the uterine body, endometrial tumor, left adnexa, endometriosis, left uterosacral ligament, uterosacral ligament, ureter, cervix, intra-abdominal adhesions]","Intra-abdominal adhesions, Adenomyosis of the uterine body, Deep infiltrating endometriosis of the left parametrium, Right parametric endometriosis, Endometrial tumor at the cervix, Endometriosis of the left adnexa",Condition after 4 surgical procedures. Intra-abdominal adhesions. Adenomyosis of the uterine body. Deep infiltrating endometriosis of the left parametrium. In the area of the left ureter and the left uterosacral ligament. Right parametric endometriosis. In the area of the right ureter and the right uterosacral ligament. Endometrial tumor at the cervix. Endometriosis of the left adnexa.,"[cervical tumor, peritoneal cavity, tumor, endometriosis, left, adhesions]","Release of adhesions, Removal of endometriosis from the left and right parametrium, Laparoscopic hysterectomy, Adhesion prevention, Drainage of the peritoneal cavity",Release of adhesions. Removal of endometriosis from the left and right parametrium. Laparoscopic hysterectomy with a cervical tumor and endometriosis-affected left and right adnexa. Adhesion prevention. Drainage of the peritoneal cavity.,Nie,NaN,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,Nie,Nie,NaN,Ćwiczę więcej niż 3 razy w tygodniu i/lub średnio robię co najmniej 10 tys kroków dziennie.,Nie,NaN,NaN,Tak,Tak,"[hypertension, lung tumors, brain]","Hypertension, brain cysts, lung tumors","Hypertension, brain cysts, lung tumors",4,Nie,Od 2 lat do 3 lat,wartość prawidłowa,0,40-49 lat
1,"[peritoneal, abdomen, endometriosis, peritoneal diseases, pain]","Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis","Pain localized in other parts of the lower abdomen, peritoneal diseases, endometriosis.","[abdominal cavity, peritoneal tissues, left sacrouterine ligament, adhesions, anterior wall, peritoneum]","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity","Excision of peritoneal tissues, release of adhesions, removal of the left sacrouterine ligament with a fragment of the peritoneum from the Douglas pouch, removal of the peritoneum from the anterior wall of the abdominal cavity.",Nie,NaN,NaN,NaN,laparoskopia,Tak -> ciągle do pół roku po operacji,Tak,Ciągle do pół roku po operacji,NaN,Tak,Byłam kilka razy u osteopaty/fizjoterapeuty w okresie do pół roku po operacji.,Ćwiczę 3 razy w tygodniu i/lub średno robię więcej niż 7 tys kroków dziennie.,Tak,"psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation","psychotherapy, acupuncture, yoga, physiotherapy, medical marijuana, meditation",Tak,Tak,"[hypothyroidism, chronic hepatitis]","Hypothyroidism, Chronic hepatitis","hypothyroidism, chronic hepatitis",2,Nie,Do pół roku,wartość prawidłowa,0,30-39 lat
2,"[bladde

In [31]:
# Sprawdzanie typu danych w komórce (wiersz 2, kolumna 'aaa')
cell_type = type(data.loc[2, 'Tlumaczenie_Ang_Rodzaje_dolegliwosci_entities'])

# Wyświetlenie typu danych
print(cell_type)

<class 'list'>


Powyższy wynik wygląda dość dobrze. Jeszcze zapiszę to do pliku żeby lepiej przyjrzeć się wynikowi.

In [32]:
# Zapis do pliku json (nie do csv bo wtedy po ponownym otwarciu listy potraktuje mi jak teksty)
data.to_json('C:/Users/justy/Documents/Projekt/03_projekt_dane_wyciągniete_NER_i_listy_z_opisow.json', orient='records')

In [33]:
# Zapis do pliku csv żeby otworzyć go sobie w MS Excel i przyjrzeć się wynikom
data.to_csv("C:/Users/justy/Documents/Projekt/03_projekt_dane_wyciągniete_NER_i_listy_z_opisow.csv", index=False)